In [1]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import re
import plotly

import csv
import statistics
import os
from tqdm.notebook import tqdm

from IPython.display import display, HTML

In [2]:
def get_csv_data_for_merge(filepath):
    all_named_kernels = {}
    stat_map = {}
    apps = []
    cached_apps = []
    configs = []
    cached_configs = []
    stats = []
    gpgpu_build_num = None
    gpgpu_build_nums = set()
    accel_build_num = None
    accel_build_nums = set()
    data = {}
    with open(filepath, "r") as data_file:
        reader = csv.reader(data_file)  # define reader object
        state = "start"
        for row in reader:  # loop through rows in csv file
            if len(row) != 0 and row[0].startswith("----"):
                state = "find-stat"
                continue
            if state == "find-stat":
                current_stat = row[0]
                stats.append(current_stat)
                state = "find-apps"
                continue
            if state == "find-apps":
                apps = row[1:]
                state = "process-cfgs"
                continue
            if state == "process-cfgs":
                if len(row) == 0:
                    if any_data:
                        cached_apps = apps
                        cached_configs = configs
                        for config in configs:
                            count = 0
                            for appargs_kname in apps:
                                first_delimiter = appargs_kname.find("--")
                                appargs = appargs_kname[:first_delimiter]
                                kname = appargs_kname[first_delimiter + 2 :]
                                if (
                                    current_stat == "GPGPU-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    gpgpu_build_num = data[config][count][21:28]
                                    gpgpu_build_nums.add(gpgpu_build_num)
                                if (
                                    current_stat == "Accel-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    accel_build_num = data[config][count][16:23]
                                    accel_build_nums.add(accel_build_num)
                                stat_map[
                                    kname + appargs + config + current_stat
                                ] = data[config][count]
                                count += 1
                    apps = []
                    configs = []
                    data = {}
                    state = "start"
                    any_data = False
                    continue
                else:
                    any_data = True
                    if accel_build_num != None and gpgpu_build_num != None:
                        full_config = (
                            row[0]
                            # + "-accel-"
                            # + str(accel_build_num)
                            # + "-gpgpu-"
                            # + str(gpgpu_build_num)
                        )
                    else:
                        full_config = row[0]
                    configs.append(full_config)
                    data[full_config] = row[1:]

    app_and_args = []
    for appargs_kname in cached_apps:
        first_delimiter = appargs_kname.find("--")
        appargs = appargs_kname[:first_delimiter]
        kname = appargs_kname[first_delimiter + 2 :]
        if appargs not in all_named_kernels:
            all_named_kernels[appargs] = []
            app_and_args.append(appargs)
        all_named_kernels[appargs].append(kname)

    # The assumption here is that every entry in each stats file is run with the same
    # git hash number, if not we are just going to warn and fail.
    if len(gpgpu_build_nums) > 1 or len(accel_build_nums) > 1:
        exit(
            "File {0} contains more than one gpgpu_build_num or accelsim_build_num - this assumes one stats file has one build num: gpgpu: {1}"
            + " accel: {2}".format(filepath, gpgpu_build_nums, accel_build_nums)
        )
    return (
        all_named_kernels,
        stat_map,
        app_and_args,
        cached_configs,
        stats,
        gpgpu_build_nums,
    )

def get_relaunch(file_name):
    _relaunch_map = {}
    file = open("./data/" + sets[0] + file_name, "r")
    # for each line in the file
    for line in file:
        segments = line.split("/")
        wl = segments[1]
        if wl == "gpgpu-sim-builds":
            continue
        print(segments)
        app_arg = segments[1] + "/" + segments[2]
        config = segments[3]
        print(segments[4])
        relaunch = segments[4].split(":")[1].replace("\n", "")
        _relaunch_map[app_arg+config] = relaunch
    return _relaunch_map

def normalize(df, config, group):
    normalized_df = pd.DataFrame()

    for c_app in df[group].unique():
        same_config = df[df[group] == c_app]
        normalized_to = same_config[same_config["g_config"] == config]

        same_config_num = same_config.select_dtypes(include=[np.number])
        # all the columns that are not numbers
        same_config_not_num = same_config.select_dtypes(exclude=[np.number])
        normalized_to_num = normalized_to.select_dtypes(include=[np.number])
        if(normalized_to_num.empty):
            normalized_to_num = pd.DataFrame(0, index=[0], columns=same_config_num.columns) 
        normalized_nums = same_config_num.div(normalized_to_num.iloc[0]).replace([np.inf, -np.inf], np.nan).fillna(0)

        normalized_same_app = pd.concat([same_config_not_num, normalized_nums], axis=1)
        normalized_df = pd.concat([normalized_df, normalized_same_app])


    assert(normalized_df.shape[0] == df.shape[0])
    assert(normalized_df.shape[1] == df.shape[1])
    return normalized_df

def get_base_kernels(filepath, base_kernels):
    stat_map = {}
    apps = []
    configs = []
    stats = []
    gpgpu_build_num = None
    gpgpu_build_nums = set()
    accel_build_num = None
    accel_build_nums = set()
    data = {}
    with open(filepath, "r") as data_file:
        reader = csv.reader(data_file)  # define reader object
        state = "start"
        for row in reader:  # loop through rows in csv file
            if len(row) != 0 and row[0].startswith("----"):
                state = "find-stat"
                continue
            if state == "find-stat":
                current_stat = row[0]
                stats.append(current_stat)
                state = "find-apps"
                continue
            if state == "find-apps":
                apps = row[1:]
                state = "process-cfgs"
                continue
            if state == "process-cfgs":
                if len(row) == 0:
                    if any_data:
                        cached_apps = apps
                        cached_configs = configs
                        for config in configs:
                            count = 0
                            for appargs_kname in apps:
                                first_delimiter = appargs_kname.find("--")
                                appargs = appargs_kname[:first_delimiter]
                                kname = appargs_kname[first_delimiter + 2 :]
                                if (
                                    current_stat == "GPGPU-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    gpgpu_build_num = data[config][count][21:28]
                                    gpgpu_build_nums.add(gpgpu_build_num)
                                if (
                                    current_stat == "Accel-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    accel_build_num = data[config][count][16:23]
                                    accel_build_nums.add(accel_build_num)
                                stat_map[
                                    kname + appargs + config + current_stat
                                ] = data[config][count]
                                if (config == "ORIN-SASS-VISUAL" or config == "ORIN-SASS") and data[config][count] != "NA" and current_stat == "gpu_tot_sim_cycle\s*=\s*(.*)":
                                    if appargs not in base_kernels:
                                        base_kernels[appargs] = []
                                    base_kernels[appargs].append(kname)
                                    # print(kname)
                                count += 1
                    apps = []
                    configs = []
                    data = {}
                    state = "start"
                    any_data = False
                    continue
                else:
                    any_data = True
                    if accel_build_num != None and gpgpu_build_num != None:
                        full_config = (
                            row[0]
                            # + "-accel-"
                            # + str(accel_build_num)
                            # + "-gpgpu-"
                            # + str(gpgpu_build_num)
                        )
                    else:
                        full_config = row[0]
                    configs.append(full_config)
                    data[full_config] = row[1:]
    return base_kernels

# workloads = ["sponza_2k", "sponza_4k"]
# workloads = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k"]
workloads = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k", "sponza_2k", "sponza_4k", "materials_2k", "materials_4k", "platformer_2k", "platformer_4k", 
            #  'demo_2k', 'demo_4k'
             ]
wl_to_name = {
    "pbrtexture_2k": "PT2",
    "pbrtexture_2k_lod0": "PT2_LOD0",
    "pbrtexture_4k": "PT4",
    "instancing_2k": "IT2",
    "instancing_2k_lod0": "IT2_LOD0",
    "instancing_4k": "IT4",
    "render_passes_2k": "SPL2",
    "render_passes_2k_lod0": "SPL2_LOD0",
    "render_passes_4k": "SPL4",
    "sponza_2k": "SPH2",
    "sponza_4k": "SPH4",
    "materials_2k": "MT2",
    "materials_4k": "MT4",
    "platformer_2k": "PL2",
    "platformer_4k": "PL4",
}
wl_to_name_no_res = {
    "pbrtexture": "PT",
    "instancing": "IT",
    "render_passes": "SPL",
    "sponza": "SPH",
    "materials": "MT",
    "platformer": "PL",
}

def config_to_name(name):
    name = name.replace("ORIN-SASS", "")
    name = name.replace("-VISUAL", "")
    name = name.replace("-concurrent", "con")
    name = name.replace("-dynamic_sm3", "-55")
    name = name.replace("-invalidate_l2", "-L2")
    name = name.replace("-best", "-sch")
    name = name.replace("-utility", "-u")
    name = name.replace("-slicer", "-sl")
    if not "con" in name:
        name = "seq" + name
    return name


    
patterns = ['', '/', '\\', 'x', '-', '|', '+', '.']

wl = 'pbrtexture_2k'
sets = [
        # 'data_baseline/',
        'all/',
        # 'correlation-vertex-l1d/',
        # 'correlation/',
        # 'data_perfl2/',
        # 'data_perfl1/',
        # "3070_baseline/"
        ]

sim_path = '/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/'
(
    all_named_kernels,
    stat_map,
    apps_and_args,
    configs,
    stats,
    gpgpu_build_nums,
) = get_csv_data_for_merge(sim_path + sets[0] + wl + ".csv")

base_kernels = {}
# for wl in workloads:
    # base_kernels = get_base_kernels(sim_path + sets[0] + wl + ".csv", base_kernels)
# base_kernels

# stats
stats

['Accel-Sim-build',
 'GPGPU-Sim-build',
 'relaunch_g',
 'relaunch_c',
 'gpgpu_n_tot_w_icount\\s*=\\s*(.*)',
 'gpu_tot_sim_insn\\s*=\\s*(.*)',
 'gpu_tot_sim_cycle\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[HIT\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_W\\]\\[HIT\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_W\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_W\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[HIT\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[HIT_RESERVED\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[MSHR_HIT\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[TO

In [3]:
# df from csv
hw = pd.read_csv('/home/tgrogers-raid/a/pan251/accel-sim-framework/hw_run/renderdoc_profiling/pbrtexture_2k.csv')

hw_cycle = pd.read_csv('/home/tgrogers-raid/a/pan251/accel-sim-framework/util/graphics/sponza_2k_cycle.csv', names=['EID', 'Cycle'])

# drop first 392 rows
# hw = hw.iloc[391:]

# merge if EID is the same
# hw = hw.merge(hw_cycle, on='EID')

hw

,EID,l1tex__lsuin_requests.sum,l1tex__texin_requests.sum,sm__inst_executed_pipe_alu.sum,sm__inst_executed_pipe_fma.sum,sm__inst_executed_pipe_lsu.sum,sm__inst_executed_pipe_tex.sum,sm__inst_issued.sum,sm__thread_inst_executed_pipe_alu_pred_on.sum,sm__thread_inst_executed_pipe_fma_pred_on.sum,...,gpu__time_duration.avg,l1tex__average_t_sector_lookup_hit.avg.ratio,l1tex__average_t_sector_pipe_lsu_lookup_hit.avg.ratio,l1tex__average_t_sector_pipe_lsu_mem_global_op_ld_lookup_hit.avg.ratio,l1tex__t_sector_hit_rate.avg.ratio,lts__average_t_sector_srcnode_gpc_lookup_hit.avg.ratio,lts__average_t_sector_srcnode_gpc_op_read_lookup_hit.avg.ratio,lts__average_t_sector_srcunit_tex_op_read.avg.ratio,lts__average_t_sector_srcunit_tex_op_read_lookup_hit.avg.ratio,lts__t_sector_hit_rate.avg.ratio
0,18,7.0,772070.0,386609.0,2899534.0,7.0,96650.0,5129980.0,12349600.0,92621500.0,...,48120.37037,0.86755,0.0,0.0,0.86755,0.61532,0.32977,0.69621,0.31272,0.61612
1,23,20328.0,1061866.0,664804.0,8967799.0,20328.0,208323.0,11630800.0,15919700.0,264482000.0,...,112703.70370,0.79246,0.0,0.0,0.79246,0.72969,0.66640,0.85053,0.61372,0.72979
2,32,42.0,5474.0,27.0,13264.0,42.0,1324.0,27994.0,704.0,418752.0,...,10425.92593,0.96107,0.0,0.0,0.96107,0.65106,0.36472,0.07890,0.01631,0.71543
3,34,196.0,2680.0,126.0,6722.0,196.0,661.0,15130.0,3936.0,200672.0,...,20703.70370,0.60671,0.0,0.0,0.60671,0.81589,0.50139,0.31114,0.19390,0.85071


In [4]:

def get_thread_count_wl(path):
    files = os.listdir(path)

    thread_count = pd.DataFrame(columns=['index', 'thread_count'])

    for file_name in files:
        if '.g' in file_name:
            continue
        with open(path + file_name, "r") as file:
                                index = int(file_name.split('_')[-1].split('.')[0])

                                grid_x = 0
                                block_x = 0
                                for line in file:
                                    # -grid dim = (585,1,1)
                                    # -block dim = (64,1,1)
                                    match = re.match(r"\-grid dim = \((\d*),(\d*),(\d*)\)", line)
                                    if match:
                                        grid_x = int(match.group(1))
                                    match = re.match(r"\-block dim = \((\d*),(\d*),(\d*)\)", line)
                                    if match:
                                        block_x = int(match.group(1))
                                    if grid_x != 0 and block_x != 0:
                                        # print(index, grid_x, block_x)
                                        count = grid_x * block_x
                                        if count <= 64:
                                            count = 0
                                        thread_count = thread_count.append({'index': index, 'thread_count': count}, ignore_index=True)
                                        break

    thread_count.sort_values(by='index', inplace=True)
    thread_count.reset_index(drop=True, inplace=True)
    thread_count.drop(columns='index')
    return thread_count

In [6]:
def plot(hw, sim, hw_col, sim_col, title):
    # correl_co = np.corrcoef(hw[hw_col].to_list(), sim[sim_col].to_list())[0][1]
    # hw_copy = hw[hw_col].copy(deep=True)
    # for i in range(len(hw_copy)):
    #     if hw_copy[i] == 0:
    #         hw_copy[i] = sim[sim_col][i]
    # mae = np.mean(np.abs((hw[hw_col] - sim[sim_col]) / hw_copy)) * 100
    moe_annote = []
    symbals = ['circle', 'x']

    fig = go.Figure()
    count = 0
    for app in sim['label'].unique():
        correl_co = np.corrcoef(hw[hw['label'] == app][hw_col].to_list(), sim[sim['label'] == app][sim_col].to_list())[0][1]
        hw_copy = hw[hw['label'] == app][hw_col].copy(deep=True)
        for i in range(len(hw_copy)):
            if hw_copy.iloc[i] == 0:
                hw_copy.iloc[i] = sim[sim['label'] == app][sim_col].iloc[i]
        mae = np.mean(np.abs((hw[hw['label'] == app][hw_col] - sim[sim['label'] == app][sim_col]) / hw_copy)) * 100
        # moe_annote.append("{0}: Correl={1:.4f} MAE={2:.4f}".format(app, correl_co, mae))
        # annot = "{0}: Corr={1:.2f}%".format(app, correl_co * 100)
        annot = "{0}: MAPE={1:.2f}%".format(app, mae)
        moe_annote.append(annot)
        fig.add_trace(
            go.Scatter(
                x=hw[hw['label'] == app][hw_col],
                y=sim[sim['label'] == app][sim_col],
                mode="markers",
                marker=dict(size=10, symbol=symbals[count % 2]),
                name=annot,
                text=annot
            )
        )
        count += 1
    
    fig.update_layout(
        # title=title,
        xaxis_title="HW " + title,
        yaxis_title="Sim " + title,
    )
    fig.update_layout(showlegend=True)

    annote = ""
    for i in range(len(moe_annote)):
        annote += moe_annote[i]
        if i != len(moe_annote) - 1:
            # newline
            annote += "<br>"
    # top left
    # fig.add_annotation(
    #     x=0,
    #     y=1.2,
    #     xref="paper",
    #     yref="paper",
    #     text=annote,
    #     showarrow=False,
    #     font=dict(size=30, family='sans-serif'),
    #     # left align
    #     align="left",
    # )

    # draw 1
    fig.add_shape(
        dict(
            type="line",
            x0=1,
            y0=1,
            x1=hw[hw_col].values.max(),
            y1=hw[hw_col].values.max(),
            line=dict(color="Red", width=1),
        )
    )
    # margin
    fig.update_layout(
        xaxis=dict(
            titlefont=dict(size=25, color="black",family='sans-serif'),
            tickfont=dict(size=20, color="black",family='sans-serif'),
            type="log",
            autorange=True,
            gridcolor="gainsboro"
        ),
        yaxis=dict(
            title="Sim " + title,
            titlefont=dict(size=25, color="black", family='sans-serif'),
            tickfont=dict(size=20, color="black", family='sans-serif'),
            type="log",
            autorange=True,
            gridcolor="gainsboro"
        ),
        width=800, height=400,
        title_font_family="sans-serif",
        title_font_size=25,
        margin=dict(l=20, r=10, t=30, b=20),
        # legend to top
        legend=dict(
            yanchor="top",
            y=1.1,
            xanchor="left",
            x=0.01,
            font=dict(size=25, family='sans-serif'),
            # orientation="h",
            traceorder="reversed",
        ),
        plot_bgcolor="white",
    )
    # backgroun white with grid
    # fig.update_layout(
    #     plot_bgcolor="white",
    #     xaxis=dict(gridcolor="gainsboro"),
    #     yaxis=dict(gridcolor="gainsboro"),
    # )

    if "rate" in title:
        fig.update_xaxes(type="linear")
        fig.update_yaxes(type="linear")
    fig.show()
    return fig

sim = pd.DataFrame(columns=[
    'app', 'label', 'config',
    'drawcall', 'l1_tex_access', 'l1_tex_hit', 'l1_global_access','cycle', 'vs', 'fs', 'l2_tex_read', 'l2_tex_hit', 'tot_cycle'])
hw  = None
sim_count = []

base_count_g = {}
base_count_c = {}
for app, kernels in base_kernels.items():
    base_count_g[app] = 0
    base_count_c[app] = 0
    for k in kernels:
        if "MESA" in k:
            base_count_g[app] += 1
        else:
            base_count_c[app] += 1
base_count_g, base_count_c

drawcall = 0

for dataset in sets:
    for wl in [
        # 'instancing_2k_lod0',
        # 'pbrtexture_2k_lod0',
        # 'render_passes_2k_lod0',
        'instancing_2k',
        'pbrtexture_2k',
        'render_passes_2k',
        'sponza_2k',  
        'materials_2k',
        'platformer_2k',
        'instancing_4k',
        'pbrtexture_4k',
        'render_passes_4k',
        'sponza_4k',
        'materials_4k',
        'platformer_4k',
         ]:
        count = 0
        print(sim_path + dataset + wl + ".csv")
        (
            all_named_kernels,
            stat_map,
            apps_and_args,
            configs,
            stats,
            gpgpu_build_nums,
        )= get_csv_data_for_merge(sim_path + dataset + wl + ".csv")
        # thread_count = get_thread_count_wl('/home/tgrogers-raid/a/pan251/accel-sim-framework/hw_run/traces/vulkan/{0}/NO_ARGS/traces/'.format(wl))
        thread_count = 0
        last_kernel = ''
        for app in apps_and_args:
            if 'NO_ARGS' not in app:
                continue
            for config in configs:
                # if config != 'RTX3070-SASS-concurrent-fg-VISUAL':
                if config != 'ORIN-SASS-VISUAL':
                    continue
                for kernel in all_named_kernels[app]:
                    # print(kernel)
                    if "VERTEX" in kernel:
                        drawcall += 1
                    kernel_index = int(kernel.split('-')[-1]) - 1
                    if (wl == 'pbrtexture_2k'):
                        if(kernel_index >= 2):
                            break
                    # thread_c = thread_count.loc[kernel_index, 'thread_count']
                        
                    stat = '\\s+Total_core_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)'
                    l1_tex_read = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))

                    stat = '\\s+Total_core_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)'
                    l1_tex_miss = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))

                    stat = '\\s+Total_core_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[HIT\\]\\s*=\\s*(.*)'
                    l1_tex_hit = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))
                    # l1_tex_hit = int(l1_tex_read) - int(l1_tex_miss)
                    # l1_tex_read = int(l1_tex_read) + int(l1_tex_miss)

                    stat = 'gpu_sim_cycle\\s*=\\s*(.*)'
                    cycle = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))

                    stat = 'gpu_tot_sim_cycle\\s*=\\s*(.*)'
                    tot_cycle = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))
                    
                    stat = '\\s+Total_core_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)'
                    l1_global_access = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))

                    stat = '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)'
                    l2_tex_read = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))

                    stat = '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)'
                    l2_tex_miss = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", "0"))

                    stat = '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[HIT\\]\\s*=\\s*(.*)'
                    l2_tex_hit = (stat_map.get(kernel + app + config + stat, "NA").replace("NA", 
                    "0"))
                    
                    l2_tex_hit = int(l2_tex_read) - int(l2_tex_miss)
                    
                    name = wl
                    if 'lod0' in wl:
                        label = "LoD OFF"
                    else:
                        label = "LoD ON"
                    label = 'Vertex Count'
                    if (drawcall not in sim['drawcall'].values):
                        count += 1
                        # sim = sim.append({
                        #     'app': app,'drawcall': drawcall, 'l1_tex_access': 0, 'l1_tex_hit': 0, 
                        #     'l1_global_access': 0, 'cycle': 0, 'l2_tex_read': 0, 'l2_tex_hit': 0
                        #     }, ignore_index=True)
                        sim = sim.append(pd.Series(0, index=sim.columns), ignore_index=True)
                        sim.iloc[-1, sim.columns.get_loc('app')] = name
                        sim.iloc[-1, sim.columns.get_loc('label')] = label
                        sim.iloc[-1, sim.columns.get_loc('drawcall')] = drawcall
                        sim.iloc[-1, sim.columns.get_loc('config')] = config

                    # update all sim data
                    sim.loc[sim['drawcall'] == drawcall, 'l1_tex_access'] += int(l1_tex_read)
                    sim.loc[sim['drawcall'] == drawcall, 'l1_tex_hit'] += int(l1_tex_hit)
                    sim.loc[sim['drawcall'] == drawcall, 'l1_global_access'] += int(l1_global_access)
                    sim.loc[sim['drawcall'] == drawcall, 'cycle'] += int(cycle)
                    sim.loc[sim['drawcall'] == drawcall, 'l2_tex_read'] += int(l2_tex_read)
                    sim.loc[sim['drawcall'] == drawcall, 'l2_tex_hit'] += int(l2_tex_hit)
                    sim.loc[sim['drawcall'] == drawcall, 'tot_cycle'] = max(int(tot_cycle), sim.loc[sim['drawcall'] == drawcall, 'tot_cycle'].values[0])

                    # if 'VERTEX' in kernel:
                    #     sim.loc[sim['drawcall'] == drawcall, 'vs'] = thread_c
                    # elif 'FRAGMENT' in kernel:
                    #     sim.loc[sim['drawcall'] == drawcall, 'fs'] = thread_c

        # hw_prof = pd.read_csv('/home/tgrogers-raid/a/pan251/accel-sim-framework/hw_run/renderdoc_profiling/{0}.csv'.format(wl.replace("_lod0", "")).replace("4k", "2k"))
        # if('sponza_2k' in wl):
        #     hw_prof = hw_prof.iloc[391:]
        # if('pbrtexture_2k' in wl):
        #     hw_prof = hw_prof.iloc[1:]
        # if('demo_2k' in wl):
        #     hw_prof = hw_prof.iloc[4:]
        # if('materials_2k' in wl):
        #     hw_prof = hw_prof.iloc[4:]
        # if('instancing_2k' in wl):
        #     hw_prof = hw_prof.iloc[1:]
        # if('platformer_2k' in wl):
        #     hw_prof = hw_prof.iloc[2237:]
        # hw_prof = hw_prof[:count]
        # if hw is None:
        #     hw = hw_prof
        # else:
        #     hw = hw.append(hw_prof, ignore_index=True)

sim = sim.fillna(0)
sim['l1_tex_hitrate'] = sim['l1_tex_hit'] / sim['l1_tex_access']
sim['l2_tex_hitrate'] = sim['l2_tex_hit'] / sim['l2_tex_read']

# hw['l1_tex_hits'] = hw["l1tex__t_sector_hit_rate.avg.ratio"] * hw["l1tex__texin_requests.sum"]
# hw['l1_tex_miss'] = hw["l1tex__texin_requests.sum"] - hw['l1_tex_hits']
# hw['l2_access'] = hw['l1_tex_miss']
# hw['l2_tex_hitrate'] = hw['lts__average_t_sector_srcunit_tex_op_read_lookup_hit.avg.ratio'] * hw['lts__t_sector_hit_rate.avg.ratio'] / hw['lts__average_t_sector_srcunit_tex_op_read.avg.ratio']
# hw['l2_tex_read'] = hw['l2_access'] * hw['l2_tex_hitrate']
# hw['label'] = sim['label'].to_list()

# hw.fillna(0, inplace=True)
# assert(hw.shape[0] == sim.shape[0])
# for index,row in sim.iterrows():
#     if row['vs'] / hw.iloc[index]['sm__threads_launched_shader_vs.sum'] > 4:
#         sim.loc[index, 'vs'] = 0
# fig = plot(hw, sim,"l1tex__texin_requests.sum", "l1_tex_access", "L1 TEX Request")
# fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("l1_tex_lod"), format="pdf")
# plot(hw, sim,"sm__inst_executed_pipe_tex.sum", "l1_tex_access", "L1 TEX Inst")
# plot(hw, sim, 'l1_tex_hits', 'l1_tex_hit', "L1 TEX Hit")
# plot(hw, sim, 'l2_tex_read', 'l2_tex_hit', "L2 Hit")
# plot(hw, sim, 'l2_access', 'l2_tex_read', "L2 Read")
# plot(hw, sim, 'sm__threads_launched_shader_ps.sum', 'fs', "Pixel Shader Count")
# fig = plot(hw, sim, 'sm__threads_launched_shader_vs.sum', 'vs', "Vertex Shader Count")
# fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("vs_count"), format="pdf")
# plot(hw, sim, 'gpu__time_duration.avg', 'cycle', "Cycle")


/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/instancing_2k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/pbrtexture_2k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/render_passes_2k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/sponza_2k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/materials_2k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/platformer_2k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/instancing_4k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/pbrtexture_4k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/render_passes_4k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/sponza_4k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/materials_4k.csv
/home/tgrogers-raid/a/pan251/ar_vr_gpu_overleaf/data/all/platformer_4k.csv


In [83]:
for index,row in sim.iterrows():
    if row['vs'] / hw.iloc[index]['sm__threads_launched_shader_vs.sum'] > 4:
        sim.loc[index, 'vs'] = 0

In [123]:
# drop row 524

display(HTML(sim[sim['app'] == 'instancing_4k'].to_html()))


# sim[sim['app'] == 'render_passes_2k']['l2_tex_hit'].sum() / sim[sim['app'] == 'render_passes_2k']['l2_tex_read'].sum()

,app,label,config,drawcall,l1_tex_access,l1_tex_hit,l1_global_access,cycle,vs,fs,l2_tex_read,l2_tex_hit,tot_cycle,l1_tex_hitrate,l2_tex_hitrate
2112,instancing_4k,Vertex Count,ORIN-SASS-VISUAL,2113,234855,8777,2849089,291199,14016,14016,180202,152257,291203,0.037372,0.844924
2113,instancing_4k,Vertex Count,ORIN-SASS-VISUAL,2114,826595,416276,22221175,5033971,14016,14016,403715,373553,5325178,0.503603,0.925289
2114,instancing_4k,Vertex Count,ORIN-SASS-concurrent-fg-VISUAL,2115,234855,11063,2849089,509753,14016,14016,177581,147734,509757,0.047106,0.831925
2115,instancing_4k,Vertex Count,ORIN-SASS-concurrent-fg-VISUAL,2116,826595,413602,22221175,5504878,14016,14016,404574,375145,5504887,0.500368,0.927259
2116,instancing_4k,Vertex Count,RTX3070-SASS-VISUAL,2117,234855,6980,2849089,153551,14016,14016,185716,154105,153555,0.029720,0.829788
2117,instancing_4k,Vertex Count,RTX3070-SASS-VISUAL,2118,826595,339889,22221175,3600388,14016,14016,477369,442847,3753947,0.411192,0.927683
2118,instancing_4k,Vertex Count,RTX3070-SASS-concurrent-fg-VISUAL,2119,234855,6365,2849089,929650,14016,14016,180909,141072,929654,0.027102,0.779795
2119,instancing_4k,Vertex Count,RTX3070-SASS-concurrent-fg-VISUAL,2120,826595,338661,22221175,3682840,14016,14016,477069,442435,3682845,0.409706,0.927403
2120,instancing_4k,Vertex Count,ORIN-SASS-concurrent-mps_sm8-VISUAL,2121,234855,12012,2849089,499034,14016,14016,175552,148364,499038,0.051146,0.845129
2121,instancing_4k,Vertex Count,ORIN-SASS-concurrent-mps_sm8-VISUAL,2122,826595,429586,22221175,9633691,14016,14016,391978,363171,9650749,0.519706,0.926509


In [162]:
import gzip

# filename = '/home/tgrogers-raid/a/pan251/accel-sim-framework/sim_run_11.7/pbrtexture_2k/NO_ARGS/RTX3070-SASS-VISUAL/gpgpusim_visualizer__Thu-May-23-23-22-03-2024.log.gz'
# filename = '/home/tgrogers-raid/a/pan251/accel-sim-framework/sim_run_11.7/render_passes_2k/NO_ARGS/RTX3070-SASS-VISUAL/gpgpusim_visualizer__Thu-May-23-23-22-06-2024.log.gz'

filename = "{0}/{1}".format(
    "/home/tgrogers-raid/a/pan251/accel-sim-framework/sim_run_11.7/sponza_4k/hotlab/RTX3070-SASS-concurrent-mps_sm8-utility-VISUAL",
    "gpgpusim_visualizer__Sun-Jun--2-00-45-19-2024.log.gz"
)

array = pd.DataFrame(
    columns=["globalcyclecount",'cycle_counter', "tex_line", "verte_lines", "compute", "invalid", "g_count", "c_count", 'dynamic_sm_count']
)

if (filename.endswith('.gz')):
        file = gzip.open(filename, 'rt')
else:
    file = open(filename, 'r')

cycle = 0
pbar = tqdm(total=os.path.getsize(filename))

while file:
    line = file.readline()
    pbar.update(file.buffer.fileobj.tell() - pbar.n)
    if not line : break
    nameNdata = line.split(":")
    if (len(nameNdata) != 2): 
        print("Syntax error at '%s'" % line) 
    namePart = nameNdata[0].strip()
    dataPart= nameNdata[1].strip()
    if namePart == "globalcyclecount":
        cycle = int(dataPart)
        # if (cycle == 500):
            # array = array[0:0]
        array = array.append(pd.Series(0, index=array.columns), ignore_index=True)
        array.iloc[-1]['globalcyclecount'] = cycle
        array.iloc[-1]['cycle_counter'] = 500 * (array.shape[0] - 1)
        
    elif namePart == "L2Breakdown":
        data = dataPart.split(' ')
        array.iloc[-1]['tex_line'] = int(data[0])
        array.iloc[-1]['verte_lines'] = int(data[1])
        array.iloc[-1]['compute'] = int(data[2])
        array.iloc[-1]['invalid'] = int(data[3])
    elif namePart == "AvgGRThreads":
        data = float(dataPart)
        array.iloc[-1]['g_count'] = data
    elif namePart == "AvgCPThreads":
        data = float(dataPart)
        array.iloc[-1]['c_count'] = data
    elif namePart == 'dynamic_sm_count':
        data = float(dataPart)
        array.iloc[-1]['dynamic_sm_count'] = data

  0%|          | 0/459847233 [00:00<?, ?it/s]

In [161]:
fig = go.Figure()
# from plotly.subplots import make_subplots

# fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['g_count'] / (1024+512), mode='lines', 
                         hoverinfo='x+y', stackgroup='one', name='Rendering Shader'))
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['c_count'] / (1024+512), mode='lines', 
                         hoverinfo='x+y', stackgroup='one', name='Compute Kernel'))
# fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['dynamic_sm_count']/16, mode='lines', line_width=2, name='Partition Ratio'),
#                 # secondary_y=True
#                 )


fig.update_layout(
    # title='L2 Breakdown',
    xaxis_title='Global Cycle',
    yaxis_title='Occupancy',
    xaxis=dict(
        titlefont=dict(size=25, color="black", family="sans-serif"),
        tickfont=dict(size=15, color="black", family="sans-serif"),
        autorange=True,
    ),
    yaxis=dict(
        titlefont=dict(size=25, color="black", family="sans-serif"),
        tickfont=dict(size=15, color="black", family="sans-serif"),
        autorange=True,
    ),
    width=800,
    height=300,
    title_font_family="sans-serif",
    title_font_size=25,
    margin=dict(l=20, r=10, t=50, b=0),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0,
        font=dict(size=20, family="sans-serif")
        
    ),
    title=dict(
        font=dict(size=25, family="sans-serif"),
        text="PT + VIO",
        # text="Occupancy: {0}".format('73.13%'),
        x=0.98,
        y=0.95
    ),

)
# fig.add_shape(type="rect",
#     # xref="paper", yref="paper",
#     x0=3.4*1000*1000, x1=3.65*1000*1000,
#     y0=0.0, y1=0.2,
#     line=dict(
#         color="LightSeaGreen",
#         width=3,
#     ),
# )

# for index,row in array[array['globalcyclecount'] == 500].iterrows():
    # fig.add_vline(x=row['cycle_counter'], line_dash="dash", )
fig.show()
fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("slicer_occupancy"), format="pdf")

In [160]:

fig = go.Figure()
# stacked area plot
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['tex_line']/1024/32, mode='lines', 
                         hoverinfo='x+y', stackgroup='one', name='TEX'))
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['verte_lines'] / 1024/32, mode='lines',
                            hoverinfo='x+y', stackgroup='one', name='Pipeline Data'))
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['compute'] / 1024/32, mode='lines',
hoverinfo='x+y', stackgroup='one', name='Compute Data'))
# fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['invalid'], mode='lines',
                            # hover
                            # info='x+y', stackgroup='one', name='INVALID'))

fig.update_layout(
    # title='SPH+RITNET',
    xaxis_title='Cycle',
    yaxis_title='# of L2 Cache Lines',
    xaxis=dict(
        titlefont=dict(size=25, color="black", family="sans-serif"),
        tickfont=dict(size=15, color="black", family="sans-serif"),
        autorange=True,
    ),
    yaxis=dict(
        titlefont=dict(size=25, color="black", family="sans-serif"),
        tickfont=dict(size=15, color="black", family="sans-serif"),
        autorange=True,
    ),
    width=800,
    height=300,
    title_font_family="sans-serif",
    title_font_size=25,
    margin=dict(l=20, r=10, t=50, b=0),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.,
        font=dict(size=20, family="sans-serif")
        
    ),
    title=dict(
        font=dict(size=25, family="sans-serif"),
        text="PT + VIO",
        # text="Occupancy: {0}".format('73.13%'),
        x=0.98,
        y=0.95
    ),
    # change title position

)

fig.show()
fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("tap"), format="pdf")
np.average(array['tex_line']/1024/32)

0.202054579774645

In [48]:

fig = go.Figure()
# stacked area plot
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['tex_line']/1024/32, mode='lines', 
                         hoverinfo='x+y', stackgroup='one', name='TEX'))
fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['verte_lines'] / 1024/32, mode='lines',
                            hoverinfo='x+y', stackgroup='one', name='Pipeline Data'))
# fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['compute'], mode='lines',
# hoverinfo='x+y', stackgroup='one'))
# fig.add_trace(go.Scatter(x=array['cycle_counter'], y=array['invalid'], mode='lines',
                            # hoverinfo='x+y', stackgroup='one', name='INVALID'))

fig.update_layout(
    # title='L2 Breakdown',
    xaxis_title='Cycle',
    yaxis_title='# of L2 Cache Lines',
    xaxis=dict(
        titlefont=dict(size=25, color="black", family="sans-serif"),
        tickfont=dict(size=15, color="black", family="sans-serif"),
        autorange=True,
    ),
    yaxis=dict(
        titlefont=dict(size=25, color="black", family="sans-serif"),
        tickfont=dict(size=15, color="black", family="sans-serif"),
        autorange=True,
    ),
    width=800,
    height=300,
    title_font_family="sans-serif",
    title_font_size=25,
    margin=dict(l=20, r=10, t=50, b=0),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.3,
        font=dict(size=20, family="sans-serif")
        
    ),
    title=dict(
        font=dict(size=30, family="sans-serif"),
        text="L2 Hit Rate: {0}".format('90%'),
        x=0.98,
    ),

)
# 619257	0.71686069	529528	0.855102163
fig.show()
# fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("l2_breakdown_low"), format="pdf")
np.average(array['tex_line']/1024/32)

0.02031612216188444

In [8]:
wl_us_3070 = {
    "pbrtexture_2k": 100,
    "pbrtexture_4k": 190,
    "instancing_2k": 300,
    "instancing_4k": 360,
    "render_passes_2k": 1530,
    "render_passes_4k": 1560,
    "sponza_2k": 1420,
    "sponza_4k": 3100,
    "materials_2k": 460,
    "materials_4k": 880,
    "platformer_2k": 720,
    "platformer_4k": 1160,
    "demo_2k": 220, 
    "demo_4k": 470
}
import plotly.express as px
color_array = px.colors.qualitative.Plotly

cycles = pd.DataFrame(columns=['app', 'hw', 'sim'])

markers = {
    '2k': 'circle',
    '4k': 'x'
}

colors = {
    'pbrtexture': color_array[0],
    'instancing': color_array[1],
    'rp': color_array[2],
    'sponza': color_array[3],
    'materials': color_array[4],
    'platformer': color_array[5],
    'demo': color_array[6]
}



for wl in workloads:
    hw_cycle = wl_us_orin[wl] * 1.5 * 1000
    # hw_cycle = wl_us_3070[wl] * 1.5 * 1000
    sim_cycle = sim[sim['app'] == wl]['tot_cycle'].max()
    # if 'pbrtexture_4k' in wl:
        # sim_cycle = sim_cycle / 2
    # if 'instancing' in wl:
        # sim_cycle -= 2741925 / 1000
    if wl == 'render_passes_2k':
        continue
    cycles = cycles.append({'app':wl.replace("render_passes", 'rp'), 'hw': hw_cycle, 
                            'sim': sim_cycle}, ignore_index=True)

cycles[['workload', 'variant']] = cycles['app'].str.split('_', expand=True)

correl_co = np.corrcoef(cycles['hw'].to_list(), cycles['sim'].to_list())[0][1]
title = "Cycles"
fig = go.Figure()
for workload in cycles['workload'].unique():
    for i, variant in enumerate(cycles['variant'].unique()):
        show_legend = True if i != 0 else False
        filtered_df = cycles[(cycles['workload'] == workload) & (cycles['variant'] == variant)]
        print_name = (workload).replace("rp", "render_passes")
        fig.add_trace(go.Scatter(
            x=filtered_df['hw'],
            y=filtered_df['sim'],
            mode='markers',
            name=wl_to_name_no_res[print_name] if show_legend else None,
            marker=dict(
                symbol=markers[variant],
                color=colors[workload],
                size=10
            ),
            showlegend=show_legend
        ))
# fig.add_trace(go.Scatter(x=cycles['hw'], y=cycles['sim'], text=cycles['app'] ,
                        #  mode='markers', name="Correlation: {0:.4f}".format(correl_co)))
# log scale
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.add_shape(
        dict(
            type="line",
            x0=cycles[['hw', 'sim']].values.min() / 20,
            y0=cycles[['hw', 'sim']].values.min() / 20,
            x1=cycles[['hw', 'sim']].values.max(),
            y1=cycles[['hw', 'sim']].values.max(),
            line=dict(color="Red", width=1),
        )
    )
fig.update_layout(
    title="Correlation: {0:.2f}%".format(correl_co * 100),
    xaxis_title="HW " + title,
    yaxis_title="Sim " + title,
)
    # margin
fig.update_layout(
    xaxis=dict(
        titlefont=dict(size=25, color="black",family='sans-serif'),
        tickfont=dict(size=20, color="black",family='sans-serif'),
        type="log",
        # autorange=True,
        gridcolor="gainsboro"
    ),
    yaxis=dict(
        title="Sim " + title,
        titlefont=dict(size=25, color="black", family='sans-serif'),
        tickfont=dict(size=20, color="black", family='sans-serif'),
        type="log",
        # autorange=True,
        gridcolor="gainsboro"
    ),
    width=800, height=400,
    title_font_family="sans-serif",
    title_font_size=25,
    margin=dict(l=20, r=10, t=50, b=20),
    # legend to top
    legend=dict(
        yanchor="top",
        y=1.15,
        xanchor="left",
        x=0.0,
        font=dict(size=18, family='sans-serif'),
        orientation="h",
        # traceorder="reversed",
    ),
    plot_bgcolor="white",
    title=dict(
        x=0.98,
        y=0.95
    ),
)

fig.show()
# fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("cycle_correaltion"), format="pdf")

/usr/lib/python3/dist-packages/numpy/core/_methods.py:44: RuntimeWarning:

invalid value encountered in reduce

/usr/lib/python3/dist-packages/numpy/core/_methods.py:40: RuntimeWarning:

invalid value encountered in reduce



In [ ]:
cycles = pd.DataFrame(columns=['label', 'hw', 'sim'])
for wl in workloads:
    # if 'passes' in wl:
        # continue
    cycles = cycles.append({'label': wl, 'hw': wl_us_3070[wl] * 1.3, 
                            'sim': sim[sim['label'] == wl]['tot_cycle'].max() / 1000}, ignore_index=True)

correl_co = np.corrcoef(cycles['hw'].to_list(), cycles['sim'].to_list())[0][1]
title = "Kcycles"

fig.add_trace(go.Scatter(x=cycles['hw'], y=cycles['sim'], text=cycles['label'] ,
                         mode='markers', name="Correlation: {0:.4f}".format(correl_co)))

fig.show()

In [108]:
for wl in workloads:
    print(sim[sim['label'] == wl]['tot_cycle'].max(), sim[sim['label'] == wl]['cycle'].sum())

430456 430452
2114111 2114103
3602797 3602789
3752862 3752854
2395012 2394916
4533795 4533699
3321770 3321688
6807843 6807761
5485030 5484746
6915340 6915054
7202404 7201094
8387434 8386130


In [ ]:
wl_us_orin= {
    "pbrtexture_2k": 580,
    "pbrtexture_2k_lod0": "PT2_LOD0",
    "pbrtexture_4k": "PT4",
    "instancing_2k": 1510, #0.03 + 1.48,
    "instancing_2k_lod0": "IT2_LOD0",
    "instancing_4k": "IT4",
    "render_passes_2k": 750,
    "render_passes_2k_lod0": "SPL2_LOD0",
    "render_passes_4k": 0,
    "sponza_2k": 6820,
    "sponza_4k": 
    "materials_2k": "MT2",
    "materials_4k": 
    "platformer_2k": "PL2",
    "platformer_4k": 
    "demo_2k": 'AR2',
    "demo_4k": 
}

wl_us_3070 = {
    "pbrtexture_2k": 160,
    "pbrtexture_4k": 190, # 300
    "instancing_2k": 300, #330
    "instancing_4k": 330,
    "render_passes_2k": 1530,
    "render_passes_4k": 1560,
    "sponza_2k": 1420, #2650
    "sponza_4k": 3100, #4920
    "materials_2k": 460, # 760
    "materials_4k": 880, # 1490
    "platformer_2k": 720, #3000,
    "platformer_4k": 1160, #3610
    "demo_2k": 220, #280, 116 by renderdoc, full 217
    "demo_4k": 470 # 710
}

wl_us_3070_renderdoc = {
    "pbrtexture_2k": 145.4, # 147 in total
    "pbrtexture_4k": 3048.448, # 1057 
    "instancing_2k": 319.488, #330
    "instancing_4k": "IT4",
    "render_passes_2k": 788.46 # 792.6
    "render_passes_4k": 0,
    "sponza_2k": 1220.632, #2101.184
    "sponza_4k": "",
    "materials_2k": 390, # 623.744
    "materials_4k": "",
    "platformer_2k": 313 #1805, depth 1214
    "platformer_4k": "",
    "demo_2k": 116 # 217
    "demo_4k": 
}